In [5]:
import os, sys
import imageio
import numpy as np
import argparse
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch
import cv2 as cv
import glob as glob
from numpy import clip

In [13]:
data_dir = './VAE/MNIST/'

data = torch.load(os.path.join(data_dir,'generated_data.pt'))

In [25]:
data_list = [x.detach().numpy() for x in data['x']]
print(len(data_list))

10000


In [26]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
    
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

In [27]:
def load_images(image_dir):
    images = []
    for fn in os.listdir(image_dir):
        ext = os.path.splitext(fn)[1].lower()
        img_path = os.path.join(image_dir, fn)
        img = imageio.imread(img_path)
        # calculate per-channel means and standard deviations
        means = img.mean(axis=(0, 1), dtype='float64')
        stds = img.std(axis=(0, 1), dtype='float64')
        # per-channel standardization of pixels
        pixels = (img - means) / stds
        pixels = clip(pixels, -1.0, 1.0)
        images.append(pixels)
    return images

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / np.expand_dims(e_x.sum(axis=1), axis=1)   # only difference

def crop10x10(in_path, out_path):
    # mnist
    x_cors = [2,32,62,92,122,152,182,212,242,272]
    y_cors = [2,32,62,92,122,152,182,212,242,272]
    img_size = 28
    number_channel = 1

    print(out_path)
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    in_list = glob.glob(in_path + "*.png")
    count = 0
    for img_name in in_list:
        count += 1
        if (number_channel == 1):
            img = cv.imread(img_name, 0)
        else:
            img = cv.imread(img_name, 1)
        for x in x_cors:
            for y in y_cors:
                img_crop = img[x:x + img_size, y:y + img_size]
                #print(img_crop.shape)
                if (number_channel == 1):
                    h, w = img_crop.shape
                else:
                    h, w, c = img_crop.shape
                if (h != img_size) or (w != img_size):
                    print("ERROR!!!")
                    exit()

                out_name = out_path + str(count) + "_" + str(x) + "_" + str(y) + ".png"
                #print(out_name)
                cv.imwrite(out_name, img_crop)

In [28]:
def preds2score(preds, splits=10):
    scores = []
    for i in range(splits):
        part = preds[(i * preds.shape[0] // splits):((i + 1) * preds.shape[0] // splits), :]
        kl = part * (np.log(part) - np.log(np.expand_dims(np.mean(part, 0), 0)))
        kl = np.mean(np.sum(kl, 1))
        scores.append(np.exp(kl))
    return np.mean(scores), np.std(scores)


def get_inception_score(images):
    num_splits=10
    batch_size = 100
    model_dir = './models/checkpoints/mnist_model_10.ckpt'
    splits = num_splits
    inps = []
    input_transform = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    for img in images:
        img = img.astype(np.float32)
        inps.append(np.expand_dims(img, 0))
    preds = []
    #pred_acc = []
    n_batches = int(math.ceil(float(len(inps)) / float(batch_size)))
    n_preds = 0

    net = ResNet18().cuda()
    net.load_state_dict(torch.load(model_dir))
    print("load model successfully")

    for i in range(n_batches):
        sys.stdout.write(".")
        sys.stdout.flush()
        inp = inps[(i * batch_size):min((i + 1) * batch_size, len(inps))]
        inp = np.concatenate(inp, 0)
        inp = np.expand_dims(inp, axis=1)
        inp = torch.from_numpy(inp).cuda()
        outputs = net(inp)
        pred = outputs.data.tolist()
        pred = softmax(pred)
        preds.append(pred)
        n_preds += outputs.shape[0]
    preds = np.concatenate(preds, 0)
    preds = np.exp(preds) / np.sum(np.exp(preds), 1, keepdims=True)
    mean_, std_ = preds2score(preds, splits)
    return mean_, std_

In [29]:
mean, std = get_inception_score(data_list)
print('\nInception mean: ', mean)
print('Inception std: ', std)


load model successfully
....................................................................................................
Inception mean:  1.0063228150435726
Inception std:  0.0037650017228050324


In [ ]:
import math
import numpy
from keras import Sequential
from tensorflow.keras.models import load_model, model_from_json
from keras.layers import Convolution2D, Dense, Dropout, Flatten, MaxPooling2D

from MnistClassifierModelBase import MnistClassifier


class MnistClassifier06(MnistClassifier):
    def __init__(self, verbose: bool = False):
        super().__init__(verbose)
        self.model = None
        self.verbose = verbose

    def build_model(self):
        self.model = Sequential()
        self.model.add(Convolution2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
        self.model.add(Convolution2D(32, (3, 3), activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.25))

        self.model.add(Flatten())
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(10, activation='softmax'))

        self.verbose_log(self.model.summary())
        self.compile_model()

    def compile_model(self):
        self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    def fit(self, x, y, batch_size=32, epochs=30):
        history = self.model.fit(x, y, batch_size, epochs)
        return history.history['loss'], history.history['acc']

    def train_on_batch(self, x: numpy.ndarray, y: numpy.ndarray, batch_size=32):
        num_samples = x.shape[0]
        if num_samples % batch_size != 0:
            raise RuntimeWarning('Batch size does not divide number of samples exactly. '
                                 'Last set of samples will not be used for training')
        loss, accuracy = [], []
        self.verbose_log('iteration, loss, accuracy')
        for i in range(int(math.floor(num_samples / batch_size))):
            data = x[i * batch_size:(i + 1) * batch_size, :, :]
            labels = y[i * batch_size:(i + 1) * batch_size]
            batch_loss, batch_accuracy = self.model.train_on_batch(data, labels)
            loss.append(batch_loss)
            accuracy.append(batch_accuracy)
            self.verbose_log('{0:04},{1:2.4f},{2:0.4f}\n'.format(i + 1, batch_loss, batch_accuracy))
        return loss, accuracy

    def evaluate(self, x, y):
        score = self.model.evaluate(x, y)
        return score[0], score[1]

    def classify(self, x):
        y = self.model.predict(x)
        return y

    def save_model(self, save_path):
        self.model.save(save_path)

    def save_model_data(self, json_path, weights_path):
        with open(json_path, 'w') as json_file:
            json_file.write(self.model.to_json())
        self.model.save_weights(weights_path)

    @staticmethod
    def load_model(model_path) -> MnistClassifier:
        model = load_model(model_path)
        classifier = MnistClassifier06()
        classifier.model = model
        return classifier

    @staticmethod
    def load_model_from_data(json_path, weights_path) -> MnistClassifier:
        with open(json_path, 'r') as json_file:
            model = model_from_json(json_file.read())
        model.load_weights(weights_path)
        classifier = MnistClassifier06()
        classifier.model = model
        classifier.compile_model()
        return classifier

ModuleNotFoundError: No module named 'MnistClassifierModelBase'